In [1]:
import os
import torch
import gc
import logging

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()        
    torch.cuda.ipc_collect()


try:
    %run setup_paths
except:
    %run notebooks/setup_paths
    

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

logging.info(f"current dir: {os.getcwd()}")

2025-08-18 02:47:06,898 - INFO - current dir: c:\Projects\scmsim


In [2]:
%run src/datasets
%run src/models
%run src/pipeline
%run src/estimators
%run src/loaders

In [4]:
paths = PathProvider('results','products')
uidata = MovieLensData(CsvLoader("MFSim0", "products/MFSim/samples.0.csv"))

REFIT = True
model_name = 'MF40'

if REFIT or not model_exists(paths, uidata.name(), model_name):
    train_params = MFTrainParams(
        lr = 1e-2,
        gamma = 0.95,
        batch_size = 2**12,
        n_epochs  = 0,
        shuffle = True)
    
    mft = MatrixFactorizationTrain(paths, 'MF40', n_factors=40, train_params=train_params)
    mft.fit(uidata)


2025-08-18 02:47:19,757 - INFO - loading ratings
2025-08-18 02:47:20,885 - INFO - fitting model MF40; num_items=3952; num_users=6040; params=MFTrainParams(lr=0.01, gamma=0.95, batch_size=4096, n_epochs=0, shuffle=True); device=cuda
2025-08-18 02:47:22,707 - INFO - saving model at: results\MFSim0\models\MF40.0


In [8]:
paths = PathProvider('results', 'products')

model_name = 'MF40'
group_name = 'MFSim/gt.filtered.0'
method_name = f'{model_name}.IPW'

model = load_model(paths, uidata.name(), model_name)

ipw_params = [IPWParams(), IPWParams(0, True), IPWParams(0.2, True),IPWParams(2, True)]
ipwe = MFIPWEstimator(model_name, model, ipw_params)

create_estimations(paths, uidata, method_name, group_name, ipwe, reset_ids=False)


2025-08-18 02:47:35,970 - INFO - processing: 0:4096 / 6388


C:\Projects\scmsim\src\loaders.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(dir_path / paths.model_filename())


2025-08-18 02:47:38,781 - INFO - processing: 4096:6388 / 6388


,MF40.IPW,MF40.IPW.s,MF40.IPW.clp0.2.s,MF40.IPW.clp2.s,treatment_idx,resp_idx
0,-0.012417,-0.006200,-0.006200,-0.024834,1254,1515
1,-0.012748,-0.006367,-0.006367,-0.025497,166,1515
2,-0.387583,-0.193525,-0.193525,-0.775166,1254,648
3,-0.172682,-0.086323,-0.086323,-0.345364,1197,648
4,-0.381291,-0.190514,-0.190514,-0.762583,376,648
...,...,...,...,...,...,...
6383,-0.489901,-0.244883,-0.244883,-0.979801,707,1198
6384,-0.015728,-0.007857,-0.007857,-0.031457,1213,1753
6385,-0.074338,-0.037145,-0.037145,-0.148676,1114,292
6386,-0.043212,-0.021567,-0.021567,-0.086424,695,7


In [9]:
%run src/pipeline
%run src/models
%run src/utils


basee = BasicEstimator()
create_estimations(paths, uidata, 'Basic' , group_name, basee)


2025-08-18 02:47:41,314 - INFO - processing: 0:4096 / 6388
2025-08-18 02:47:42,986 - INFO - processing: 4096:6388 / 6388


,CORR,LIFT,SATE,treatment_idx,resp_idx
0,0.314324,4.685142,0.054553,1254,1515
1,0.591057,13.657434,0.184177,166,1515
2,-0.282308,1.890124,0.383039,1254,648
3,0.400956,1.712390,0.409091,1197,648
4,-0.117298,2.335222,0.585230,376,648
...,...,...,...,...,...
6383,-0.274890,1.617399,0.327025,707,1198
6384,-0.137981,2.569125,0.097598,1213,1753
6385,0.037386,2.942134,0.183561,1114,292
6386,-0.087858,2.603356,0.103253,695,7


In [10]:
%run src/utils
%run src/pipeline
import numpy as np
aaa = load_all_estimations(paths, 'MFSim0', 'MFSIM/gt.filtered.0')
np.isnan(np.nan_to_num(aaa["SATE"].to_numpy(),0)).sum()


0

In [11]:
get_sim_scores(paths, uidata, 'MFSIM/gt.filtered.0')

,name,mse,mse_pos,mse_zero
0,CORR,0.052414487613700364,0.04650955138031817,0.05315865306391375
1,LIFT,64.82714451627756,317.67350979288045,36.66563876760943
2,SATE,0.0801535585008124,0.11018678313148068,0.07390541852736958
3,MF40.IPW,0.021199253196111607,0.06809392391489243,0.01754832685935642
4,MF40.IPW.s,0.007267460709351211,0.04852266293545583,0.004405960937768334
5,MF40.IPW.clp0.2.s,0.007267460709351211,0.04852266293545583,0.004405960937768334
6,MF40.IPW.clp2.s,0.07483050033921292,0.12659105636406073,0.07002695548471381


In [28]:
if False:
    ccc = group_path = pd.read_csv(cfg.get_product_csv("MoviesCausalGPT"))
    uuu = MovieLensData(MovieLens1MLoader('ml-1M'))
    ggg = pd.read_csv(cfg.get_product_csv('MFSim/gt.0'))
    ccc = enrich_cause_indexes(ccc, uuu.info)
    ccc["GPT"] = ccc["causal_effect"]
    
    fff = pd.merge(ccc[["treatment_idx","resp_idx", "GPT"]], ggg, on=["treatment_idx", "resp_idx"], how="inner")
    fff[fff["GPT"]>3]
    fff = fff.copy()
    ggg = fff.to_csv(cfg.get_product_csv('MFSim/gt.filtered.0'))
    #group_path = cfg.get_product_csv(group_name)
    #pdf = pd.read_csv(group_path)

